# Data Exploration - Medical Volume Analysis

This notebook explores raw medical volumes (DICOM or NIfTI) and verifies preprocessing steps.

## Objectives
- Load raw volumes (DICOM or NIfTI format)
- Display slices in three standard planes (Axial, Coronal, Sagittal)
- Verify intensity distributions
- Visualize segmentation masks (if available)


In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, Optional

sys.path.insert(0, str(Path().absolute().parent))

from src.data import preprocessing


## 1. Load Raw Volume

Load a volume from either DICOM directory or NIfTI file.


In [ ]:
# Configuration: specify path to your data
# Option 1: NIfTI file
volume_path = Path("../data/samples/exp_volume_0.nii.gz")

# Option 2: DICOM directory (uncomment to use)
# volume_path = Path("../data/raw/volumes/patient_001")

# Load volume
if volume_path.is_dir():
    print(f"Loading DICOM volume from: {volume_path}")
    volume, metadata = preprocessing.load_dicom_volume(volume_path)
else:
    print(f"Loading NIfTI volume from: {volume_path}")
    volume, metadata = preprocessing.load_nifti_volume(volume_path)

print(f"\nVolume loaded successfully!")
print(f"  Shape: {volume.shape}")
print(f"  Data type: {volume.dtype}")
print(f"  Spacing: {metadata['spacing']}")
print(f"  Origin: {metadata['origin']}")
print(f"  Modality: {metadata.get('modality', 'Unknown')}")


## 2. Display Slices in Three Standard Planes

Visualize the volume in Axial, Coronal, and Sagittal planes.


In [ ]:
def display_three_planes(volume: np.ndarray, slice_indices: Optional[Tuple[int, int, int]] = None):
    """Display volume in three standard planes.
    
    Args:
        volume: 3D volume array (D, H, W)
        slice_indices: Optional tuple (axial_idx, coronal_idx, sagittal_idx)
    """
    d, h, w = volume.shape
    
    if slice_indices is None:
        axial_idx = d // 2
        coronal_idx = h // 2
        sagittal_idx = w // 2
    else:
        axial_idx, coronal_idx, sagittal_idx = slice_indices
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Axial (top-down view)
    axes[0].imshow(volume[axial_idx, :, :], cmap="gray")
    axes[0].set_title(f"Axial Slice {axial_idx}/{d-1}")
    axes[0].axis("off")
    
    # Coronal (front view)
    axes[1].imshow(volume[:, coronal_idx, :], cmap="gray")
    axes[1].set_title(f"Coronal Slice {coronal_idx}/{h-1}")
    axes[1].axis("off")
    
    # Sagittal (side view)
    axes[2].imshow(volume[:, :, sagittal_idx], cmap="gray")
    axes[2].set_title(f"Sagittal Slice {sagittal_idx}/{w-1}")
    axes[2].axis("off")
    
    plt.tight_layout()
    plt.show()

# Display middle slices
display_three_planes(volume)


In [ ]:
def analyze_intensities(volume: np.ndarray):
    """Analyze intensity distribution of the volume."""
    print("Intensity Statistics:")
    print(f"  Min: {volume.min():.2f}")
    print(f"  Max: {volume.max():.2f}")
    print(f"  Mean: {volume.mean():.2f}")
    print(f"  Median: {np.median(volume):.2f}")
    print(f"  Std: {volume.std():.2f}")
    print(f"  Percentiles:")
    print(f"    25th: {np.percentile(volume, 25):.2f}")
    print(f"    50th: {np.percentile(volume, 50):.2f}")
    print(f"    75th: {np.percentile(volume, 75):.2f}")
    print(f"    95th: {np.percentile(volume, 95):.2f}")
    
    # Histogram
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    axes[0].hist(volume.flatten(), bins=100, edgecolor="black", alpha=0.7)
    axes[0].set_xlabel("Intensity Value")
    axes[0].set_ylabel("Frequency")
    axes[0].set_title("Intensity Histogram")
    axes[0].grid(True, alpha=0.3)
    
    # Box plot
    axes[1].boxplot(volume.flatten(), vert=True)
    axes[1].set_ylabel("Intensity Value")
    axes[1].set_title("Intensity Box Plot")
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Check if values are in HU range (typical CT scan range: -1000 to 3000)
    if volume.min() >= -1000 and volume.max() <= 3000:
        print("\nValues appear to be in Hounsfield Units (HU) range.")
    else:
        print("\nValues may not be in standard HU range.")

analyze_intensities(volume)


## 4. Visualize Segmentation Mask (if available)

If you have a corresponding segmentation mask, load and visualize it.


In [ ]:
# Load mask if available
# Adjust path according to your data structure
mask_path = Path("../data/samples/exp_mask_0.nii.gz")

if mask_path.exists():
    print(f"Loading mask from: {mask_path}")
    mask, mask_metadata = preprocessing.load_nifti_volume(mask_path)
    
    print(f"\nMask loaded successfully!")
    print(f"  Shape: {mask.shape}")
    print(f"  Data type: {mask.dtype}")
    print(f"  Unique values: {np.unique(mask)}")
    print(f"  Mask coverage: {(mask > 0).sum() / mask.size * 100:.2f}%")
    
    # Verify shapes match
    if volume.shape != mask.shape:
        print(f"\nWARNING: Volume shape {volume.shape} != Mask shape {mask.shape}")
    else:
        print(f"\nVolume and mask shapes match.")
else:
    print(f"Mask not found at {mask_path}")
    print("Skipping mask visualization.")
    mask = None


In [ ]:
def display_volume_with_mask(volume: np.ndarray, mask: np.ndarray, slice_indices: Optional[Tuple[int, int, int]] = None):
    """Display volume with segmentation mask overlay."""
    d, h, w = volume.shape
    
    if slice_indices is None:
        axial_idx = d // 2
        coronal_idx = h // 2
        sagittal_idx = w // 2
    else:
        axial_idx, coronal_idx, sagittal_idx = slice_indices
    
    fig, axes = plt.subplots(3, 2, figsize=(12, 15))
    
    planes = [
        ("Axial", volume[axial_idx, :, :], mask[axial_idx, :, :], axial_idx),
        ("Coronal", volume[:, coronal_idx, :], mask[:, coronal_idx, :], coronal_idx),
        ("Sagittal", volume[:, :, sagittal_idx], mask[:, :, sagittal_idx], sagittal_idx),
    ]
    
    for row, (plane_name, vol_slice, mask_slice, idx) in enumerate(planes):
        # Volume only
        axes[row, 0].imshow(vol_slice, cmap="gray")
        axes[row, 0].set_title(f"{plane_name} - Volume (Slice {idx})")
        axes[row, 0].axis("off")
        
        # Volume + Mask overlay
        axes[row, 1].imshow(vol_slice, cmap="gray")
        axes[row, 1].imshow(mask_slice, cmap="Reds", alpha=0.5)
        axes[row, 1].set_title(f"{plane_name} - Volume + Mask (Slice {idx})")
        axes[row, 1].axis("off")
    
    plt.tight_layout()
    plt.show()

if mask is not None:
    display_volume_with_mask(volume, mask)
else:
    print("No mask available for visualization.")


## 5. Verify Preprocessing

Test preprocessing steps to ensure they work correctly.


In [ ]:
# Test preprocessing pipeline
print("Testing preprocessing pipeline...")

# Resample to isotropic spacing
resampled, new_spacing = preprocessing.resample_isotropic(
    volume, 
    metadata["spacing"], 
    target_spacing=(1.0, 1.0, 1.0)
)

print(f"\nAfter resampling:")
print(f"  Original shape: {volume.shape}")
print(f"  Resampled shape: {resampled.shape}")
print(f"  Original spacing: {metadata['spacing']}")
print(f"  New spacing: {new_spacing}")

# Normalize HU values
normalized = preprocessing.normalize_hu(resampled, hu_min=-1000.0, hu_max=1000.0)

print(f"\nAfter normalization:")
print(f"  Min: {normalized.min():.2f}")
print(f"  Max: {normalized.max():.2f}")
print(f"  Mean: {normalized.mean():.2f}")

# Auto crop
cropped, crop_slices = preprocessing.auto_crop(normalized, threshold=0.01)

print(f"\nAfter cropping:")
print(f"  Original shape: {normalized.shape}")
print(f"  Cropped shape: {cropped.shape}")
print(f"  Crop slices: {crop_slices}")

# Visualize preprocessed volume
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

d, h, w = cropped.shape
axes[0].imshow(cropped[d//2, :, :], cmap="gray")
axes[0].set_title("Preprocessed - Axial")
axes[0].axis("off")

axes[1].imshow(cropped[:, h//2, :], cmap="gray")
axes[1].set_title("Preprocessed - Coronal")
axes[1].axis("off")

axes[2].imshow(cropped[:, :, w//2], cmap="gray")
axes[2].set_title("Preprocessed - Sagittal")
axes[2].axis("off")

plt.tight_layout()
plt.show()

print("\nPreprocessing pipeline verified successfully!")


## Summary

This notebook demonstrated:
1. Loading raw volumes from DICOM or NIfTI formats
2. Displaying slices in three standard planes (Axial, Coronal, Sagittal)
3. Analyzing intensity distributions
4. Visualizing segmentation masks when available
5. Verifying preprocessing pipeline

The preprocessing steps include:
- Isotropic resampling (1x1x1 mm)
- HU normalization (clamped to [-1000, 1000])
- Automatic cropping of empty regions
